In [ ]:

def ica_setup(source_noise, source_nonG):
    """
    source_noise    :   grf generated using gaussianfield [in Notebook Setup above]
    source_nonG     :   returns n columns corresponding to n gaussian peaks that are shifted by xPeak/xc relative to 0 (and scaled by the size of the field)

    source_comps    :   array of source component arrays
    num_comps       :   num of different source signals/components, i.e. GRF & no. of peaks
    num_samples     :   num of observations (has to be >= num_comps)
    mix_matrix      :   mixing matrix generated randomly with entries over [0.5, 1)
    mix_signal      :   resulting mixed/observed signals (not prewhitened)
    """

    source_comps = np.vstack([source_nonG, source_noise])
    num_comps = source_comps.shape[0]
    num_samples = num_comps

    mix_matrix = (1.0+np.random.random((num_samples, num_comps)))/2.0
    mix_signal = np.dot(mix_matrix, source_comps) # mixed signals

    return mix_signal, source_comps, num_comps

def ica_prewhiten(mix_signal, kbin_size=None):
    """

    Handling the two observed signals separately. 
    Preprocessing involves mean subtraction and dividing by the variance (in k-space).
    """

    size = mix_signal[0, :].size
    k_size = size//2 + 1
    
    if kbin_size==None:
        nkbins = int(k_size/50)
    else:
        nkbins = int(k_size//kbin_size)
    kc = np.linspace(0, k_size, nkbins+1)
    kc_size = kc.size

    sample1_pre = mix_signal[0, :]
    sample2_pre = mix_signal[1, :]

    print(np.mean(sample1_pre), np.mean(sample2_pre))

    sample1_ft = np.fft.rfft(sample1_pre)
    sample2_ft = np.fft.rfft(sample2_pre)
    for i in range(kc_size-1):
        count = i+1
        klow = int(kc[i])
        khigh = int(kc[i+1])

        sample1_sqrtpower = np.absolute(sample1_ft[klow:khigh]) #k-space variance
        sample1_ft[klow:khigh] = sample1_ft[klow:khigh] * ( size )**(1/2) / sample1_sqrtpower  # Whitening the field
        
        sample2_sqrtpower = np.absolute(sample2_ft[klow:khigh])
        sample2_ft[klow:khigh] = sample2_ft[klow:khigh] * ( size )**(1/2) / sample2_sqrtpower

    sample1 = np.fft.irfft(sample1_ft)
    sample2 = np.fft.irfft(sample2_ft)

    print(np.mean(sample1), np.mean(sample2))
    m1 = np.mean(sample1)
    sample1 = sample1 - m1 #Subtracting the mean
    # Sample 2 - same procedure as above
    m2 = np.mean(sample2)
    sample2 = sample2 - m2
    print(np.mean(sample1), np.mean(sample2))

    # Mix the samples back again
    mix_signal = np.vstack([sample1, sample2])

    return mix_signal

def ica_run(mix, num_comps, max_iter=1e4, tol=1e-5, 
        fun='logcosh', whiten='unit-variance', algo='parallel'):
    """Initialize FastICA with given params.

    Notes:
            Logcosh is negentropy.
    """
    
    # , white='unit-variance'
    transformer = FastICA(n_components=num_comps, algorithm=algo, whiten=whiten, max_iter=max_iter, tol=tol, fun=fun)

    # run FastICA on observed (mixed) signals
    sources = transformer.fit_transform(mix.T)
    return sources.T

def ica_swap(source_comps, ica_src):
    """
    
    """
    
    # print('\nBeginning swap...')
    ica_sources = np.ndarray.copy(ica_src)

    srcng = source_comps[0, :]
    srcg = source_comps[1, :]
    ica0 = ica_sources[0, :]
    ica1 = ica_sources[1, :]

    dist_ng0 = np.linalg.norm(srcng**2 - ica0**2, 1)
    dist_ng1 = np.linalg.norm(srcng**2 - ica1**2, 1)
    dist_g0 = np.linalg.norm(srcg**2 - ica0**2, 1)
    dist_g1 = np.linalg.norm(srcg**2 - ica1**2, 1)
    # print('dist nong->ica1:', dist_ng1, ' | dist nong->ica0:', dist_ng0)
    # print('dist g->ica0:', dist_g0, ' | dist g->ica1:', dist_g1)
    if dist_ng0 > dist_ng1:
        # print('dist nong->ica1:', dist_ng1, ' | dist nong->ica0:', dist_ng0)
        # print('dist g->ica0:', dist_g0, ' | dist g->ica1:', dist_g1)
        ica_sources = np.flip(ica_sources, 0)
        print('Swapped!')

    # icang, icag = ica_sources[0, :], ica_sources[1, :]
    # dist_ngng = np.linalg.norm(srcng**2 - icang**2, 1)
    # print('dist nong->icang:', dist_ngng)
    
    # print('...ending swap.\n')
    return ica_sources

def ica_signflip(source_comps, ica_src):
    """
    
    """
    
    # print('\nBeginning flip...')
    ica_sources = np.ndarray.copy(ica_src)

    srcng = source_comps[0, :]
    srcg = source_comps[1, :]
    icang = ica_sources[0, :]
    icag = ica_sources[1, :]

    dist_gg = np.linalg.norm(srcg - icag, 1)
    dist_neg_gg = np.linalg.norm(srcg + icag, 1)
    dist_ngng = np.linalg.norm(srcng - icang, 1)
    dist_neg_ngng = np.linalg.norm(srcng + icang, 1)
    
    if dist_gg > dist_neg_gg:
        # print('dist_gg:', dist_gg, ' | dist_neg_gg:', dist_neg_gg)
        icag = -icag
        print('Gauss sign flipped!')

    if dist_ngng > dist_neg_ngng:
        # print('dist_ngng:', dist_ngng, ' | dist_neg_ngng:', dist_neg_ngng)
        icang = -icang
        print('NonG sign flipped!')

    ica_sources[0, :], ica_sources[1, :] = icang, icag
    
    # print('...ending flip.\n')
    return ica_sources

def ica_scale(source_comps, ica_src):
    """
    
    """

    ica_sources = np.ndarray.copy(ica_src)

    src_ng_max = np.abs(source_comps[0]).max()
    src_g_max = np.abs(source_comps[1]).max()
    src_max = np.abs(source_comps).max()
    ica_ng_max = np.abs(ica_sources[0]).max()
    ica_g_max = np.abs(ica_sources[1]).max()
    ica_max = np.abs(ica_sources).max()
    
    ng = ica_sources[0, :]
    ng = ng * ( src_ng_max / ica_ng_max )
    g = ica_sources[1, :]
    g = g * ( src_g_max / ica_g_max )
    ica_sources[0, :] = ng; ica_sources[1, :] = g

    ica_ng_max = np.abs(ng).max()
    ica_g_max = np.abs(g).max()
    ica_max = np.abs(ica_sources).max()

    return ica_sources, [src_max, src_ng_max, src_g_max], [ica_max, ica_ng_max, ica_g_max]

def ica_prepres(source_comps, ica_src):
    """
    
    """

    ica_sources = np.ndarray.copy(ica_src)

    ica_sources = ica_swap(source_comps, ica_sources)
    ica_sources = ica_signflip(source_comps, ica_sources)
    ica_sources, src_max, ica_max = ica_scale(source_comps, ica_sources)

    return ica_sources, src_max, ica_max

def ica(field_g, field_ng, 
            max_iter=1e4, tol=1e-5, fun='logcosh', whiten='unit-variance', algo='parallel', 
                prewhiten = False, wbin_size = None):
    """
    
    """
    
    mix_signal, src, num_comps = ica_setup(field_g, field_ng)
    if prewhiten:
        mix_signal = ica_prewhiten(mix_signal, wbin_size)

    ica_src_og = ica_run(mix_signal, num_comps, mix, num_comps, max_iter=max_iter, tol=tol, fun=fun, whiten=whiten, algo=algo)
    
    ica_src, src_max, ica_max = ica_prepres(src, ica_src_og)

    return ica_src, [src_max, ica_max]

def resid(a, b):
    """
    
    """
    
    bdota = np.dot(b, a)
    adota = np.dot(a, a)
    rv = b - (bdota / adota) * a
    r = np.linalg.norm(rv, 2)
    
    anorm = np.linalg.norm(a, 2)
    rr = r/anorm

    return r, rr

In [ ]:
"""Filtering.

"""

def filter_ica(field_g, field_ng, 
                nbins=10, (klow, khigh), 
                    max_iter=1e4, tol=1e-5, fun='logcosh', whiten='unit-variance', algo='parallel', 
                        prewhiten = False, wbin_size = None):
    """

    """

    #
    #
    # Filtering parameters/vars
    #
    #
    nbins = 10
    k_size = size//2 + 1
    k_low = 0
    kl_global = k_low
    k_high = k_size
    kc = np.linspace(0, k_high, nbins+1)
    kc_size = kc.size

    #
    #
    # ICA parameters/vars
    #
    #
    max_iter = int(9e13)
    tol = 1e-12
    ica_src = np.zeros((kc_size+1, 2, size))
    max_amps = np.zeros((kc_size+1, 2))

    #
    #
    # Run ICA
    #
    #
    ica_src[0, :], max_amps[0, :] = ica(field_g, field_ng, 
                                    max_iter=max_iter, tol=tol, fun=fun, whiten=whiten, algo=algo, 
                                        prewhiten = prewhiten, wbin_size = wbin_size)
    src_max, ica_max = max_amps[0], max_amps[1]

    for i in range(kc_size-1):
        count = i+1
        klow = kc[i]
        khigh = kc[i+1]

        print(f"\nProcessing k-bin number:    {count} ...")

        #
        #
        # Filter
        #
        #
        filtered = filter(zg, zng, size, int(klow), int(khigh))
        zgf, zngf = filtered[0], filtered[1]
        
        #
        #
        # Run ICA
        #
        #
        ica_src[count, :], max_amps[count, :] = ica(field_g, field_ng, 
                                            max_iter=max_iter, tol=tol, fun=fun, whiten=whiten, algo=algo, 
                                                prewhiten = prewhiten, wbin_size = wbin_size)
        src_max, ica_max = max_amps[0], max_amps[1]
    
    return ica_src, max_amps


In [ ]:

def plot_ica(ica_src, max_amps):
    """

    """

    #
    #
    # Plot
    #
    #
    plt.rcParams.update({'font.size': 7})
    nrows = nbins + 1
    ncols = 2

    fig, ax = plt.subplots(nrows, ncols, sharex='all', figsize=(6*ncols, 3*nrows), constrained_layout=True)

    offset = src_max[0]*1.8
    offset_ica = ica_max[0]*1.8

    ax00 = ax[0, 0]
    # Plotting source components
    ax[0, 0].set_title("(a) Source Components")
    for j in range(num_comps):
        if j == 0:
            label = "Non-Gaussian Component"
        else:
            label = "Gaussian Component"
        ax[0, 0].plot(src[j, :] + offset*j, label=label)
    ax[0, 0].set(ylabel="Zeta amplitude without filtering.")
    ax[0, 0].legend(loc=1)

    ax01 = ax[0, 1]
    # Plotting ICA-separated signals
    ax[0, 1].set_title("(b) ICA-Separated Signals")
    ax[0, 1].sharey(ax00)
    for j in range(num_comps):
        if j == 0:
            label = "Non-Gaussian Component"
        else:
            label = "Gaussian Component"
        ax[0, 1].plot(ica_src[0, j, :] + offset_ica*j, label=label) # Amplitudes are scaled arbitrarily because ICA doesn't recover amp
    # ax[0, 1].legend()

    ax[0, 0].text(0.5, 0.5, "UNFILTERED - FULL FIELD", 
                    fontsize='xx-large', transform=ax[0, 0].transAxes, 
                        ha='center', va='center', alpha=0.4)
    ax[0, 1].text(0.5, 0.5, "UNFILTERED - FULL FIELD", 
                    fontsize='xx-large', transform=ax[0, 1].transAxes, 
                        ha='center', va='center', alpha=0.4)
    ax[0, 0].legend(loc=1)

    for i in range(kc_size-1):
        count = i+1
        klow = kc[i]
        khigh = kc[i+1]

        print(f"\nProcessing k-bin number:    {count} ...")

        #
        #
        # Filter
        #
        #
        filtered = filter(zg, zng, size, int(klow), int(khigh))
        zgf, zngf = filtered[0], filtered[1]
        
        #
        #
        # Run ICA
        #
        #
        mix_signal, src, num_comps = ica_setup(zgf, zngf)
        # mix_signal = ica_preprocess(mix_signal, 100)
        ica_src_og = ica_run(mix_signal, num_comps, max_iter, tol)
        ica_src[count, :], src_max, ica_max = ica_prepres(src, ica_src_og)

        offset_ = src_max[0]*1.8
        offset_ica_ = ica_max[0]*1.8
        klow = round(klow, 1); khigh = round(khigh, 1)

        # Plotting source components
        ax[count, 0].sharey(ax00)
        for j in range(num_comps):
            if j == 0:
                label = "Non-Gaussian Component"
            else:
                label = "Gaussian Component"
            ax[count, 0].plot(src[j, :] + offset*j, label=label)
        ax[count, 0].set(ylabel=f'{i+1}) ' + "Zeta Amplitude with filter: " + r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh))
        # ax[count, 0].legend()
        
        ax[count, 1].sharey(ax00)
        # Plotting ICA-separated signals
        for j in range(num_comps):
            if j == 0:
                label = "Non-Gaussian Component"
            else:
                label = "Gaussian Component"
            ax[count, 1].plot(ica_src[count, j, :] + offset_ica*j, label=label) # Amplitudes are scaled arbitrarily because ICA doesn't recover amp
        # ax[count, 1].legend()


        

        ax[count, 0].text(0.5, 0.5, r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh), 
                                fontsize='xx-large', transform=ax[count, 0].transAxes, 
                                    ha='center', va='center', alpha=0.4)
        ax[count, 1].text(0.5, 0.5, r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh), 
                                fontsize='xx-large', transform=ax[count, 1].transAxes, 
                                    ha='center', va='center', alpha=0.4)

    ax_count = kc_size-1
    ax[ax_count, 0].set(xlabel=r'$x$')
    ax[ax_count, 1].set(xlabel=r'$x$')

    fig.suptitle(rf'Filtered $\it{{FastICA}}$-separation with $k: [{{{k_low}}}, {{{k_high}}}]$.' + f'\nField size: {size}.', fontsize=16)

    note="Note: The Gaussian components are manually offset up from 0 for the purpose of clarity."
    fig.text(0.5, -0.01, note, wrap=True, horizontalalignment='center', fontsize=8)
    plt.show()

    plt.savefig(f'/fs/lustre/cita/haider/projects/pnong_ml/ica/plots/icafiltered_s{size}_{int(kl_global)}to{int(khigh)}k{nbins}.png')

In [ ]:

    
#
#
# Filtering parameters/vars
#
#
nbins = 10
k_size = size//2 + 1
k_low = 0
kl_global = k_low
k_high = k_size
kc = np.linspace(0, k_high, nbins+1)
# kc = np.array([0, 20, 40, 80])
kc_size = kc.size

#
#
# ICA parameters/vars
#
#
max_iter = int(9e13)
tol = 1e-12
ica_src = np.zeros((kc_size+1, 2, size))

#
#
# Run ICA
#
#
mix_signal, src, num_comps = ica_setup(zg, zng)
# mix_signal = ica_preprocess(mix_signal, 100)
ica_src_og = ica_run(mix_signal, num_comps, max_iter, tol)
ica_src[0, :], src_max, ica_max = ica_prepres(src, ica_src_og)


#
#
# Plot
#
#
plt.rcParams.update({'font.size': 7})
nrows = nbins + 1
ncols = 2

fig, ax = plt.subplots(nrows, ncols, sharex='all', figsize=(6*ncols, 3*nrows), constrained_layout=True)

offset = src_max[0]*1.8
offset_ica = ica_max[0]*1.8

ax00 = ax[0, 0]
# Plotting source components
ax[0, 0].set_title("(a) Source Components")
for j in range(num_comps):
    if j == 0:
        label = "Non-Gaussian Component"
    else:
        label = "Gaussian Component"
    ax[0, 0].plot(src[j, :] + offset*j, label=label)
ax[0, 0].set(ylabel="Zeta amplitude without filtering.")
ax[0, 0].legend(loc=1)

ax01 = ax[0, 1]
# Plotting ICA-separated signals
ax[0, 1].set_title("(b) ICA-Separated Signals")
ax[0, 1].sharey(ax00)
for j in range(num_comps):
    if j == 0:
        label = "Non-Gaussian Component"
    else:
        label = "Gaussian Component"
    ax[0, 1].plot(ica_src[0, j, :] + offset_ica*j, label=label) # Amplitudes are scaled arbitrarily because ICA doesn't recover amp
# ax[0, 1].legend()

ax[0, 0].text(0.5, 0.5, "UNFILTERED - FULL FIELD", 
                fontsize='xx-large', transform=ax[0, 0].transAxes, 
                    ha='center', va='center', alpha=0.4)
ax[0, 1].text(0.5, 0.5, "UNFILTERED - FULL FIELD", 
                fontsize='xx-large', transform=ax[0, 1].transAxes, 
                    ha='center', va='center', alpha=0.4)
ax[0, 0].legend(loc=1)

for i in range(kc_size-1):
    count = i+1
    klow = kc[i]
    khigh = kc[i+1]

    print(f"\nProcessing k-bin number:    {count} ...")

    #
    #
    # Filter
    #
    #
    filtered = filter(zg, zng, size, int(klow), int(khigh))
    zgf, zngf = filtered[0], filtered[1]
    
    #
    #
    # Run ICA
    #
    #
    mix_signal, src, num_comps = ica_setup(zgf, zngf)
    # mix_signal = ica_preprocess(mix_signal, 100)
    ica_src_og = ica_run(mix_signal, num_comps, max_iter, tol)
    ica_src[count, :], src_max, ica_max = ica_prepres(src, ica_src_og)

    offset_ = src_max[0]*1.8
    offset_ica_ = ica_max[0]*1.8
    klow = round(klow, 1); khigh = round(khigh, 1)

    # Plotting source components
    ax[count, 0].sharey(ax00)
    for j in range(num_comps):
        if j == 0:
            label = "Non-Gaussian Component"
        else:
            label = "Gaussian Component"
        ax[count, 0].plot(src[j, :] + offset*j, label=label)
    ax[count, 0].set(ylabel=f'{i+1}) ' + "Zeta Amplitude with filter: " + r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh))
    # ax[count, 0].legend()
    
    ax[count, 1].sharey(ax00)
    # Plotting ICA-separated signals
    for j in range(num_comps):
        if j == 0:
            label = "Non-Gaussian Component"
        else:
            label = "Gaussian Component"
        ax[count, 1].plot(ica_src[count, j, :] + offset_ica*j, label=label) # Amplitudes are scaled arbitrarily because ICA doesn't recover amp
    # ax[count, 1].legend()

    a = src[0, :]
    b = ica_src[count, 0, :]
    bdota = np.dot(b, a)
    adota = np.dot(a, a)
    rv = b - (bdota / adota) * a
    r = np.linalg.norm(rv, 2)
    anorm = np.linalg.norm(a, 2)
    rr = r/anorm


    print("residual (input vs ouput nonG): ", rr)

    ax[count, 0].text(0.5, 0.5, r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh), 
                            fontsize='xx-large', transform=ax[count, 0].transAxes, 
                                ha='center', va='center', alpha=0.4)
    ax[count, 1].text(0.5, 0.5, r"$k=[{{{kl}}}, {{{kh}}}]$".format(kl=klow, kh=khigh), 
                            fontsize='xx-large', transform=ax[count, 1].transAxes, 
                                ha='center', va='center', alpha=0.4)

ax_count = kc_size-1
ax[ax_count, 0].set(xlabel=r'$x$')
ax[ax_count, 1].set(xlabel=r'$x$')

fig.suptitle(rf'Filtered $\it{{FastICA}}$-separation with $k: [{{{k_low}}}, {{{k_high}}}]$.' + f'\nField size: {size}.', fontsize=16)

note="Note: The Gaussian components are manually offset up from 0 for the purpose of clarity."
fig.text(0.5, -0.01, note, wrap=True, horizontalalignment='center', fontsize=8)
plt.show()

plt.savefig(f'/fs/lustre/cita/haider/projects/pnong_ml/ica/plots/icafiltered_s{size}_{int(kl_global)}to{int(khigh)}k{nbins}.png')